<a href="https://colab.research.google.com/github/aerofa45/Recommendation_System/blob/main/Rec_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

df = pd.read_csv("movie.metadata.tsv", sep = "\t")
df.head()

,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science Fiction"", ""/m/03npn"": ""Horror"", ""/m/03k9fj"": ""Adventure"", ""/m/0fdjb"": ""Supernatural"", ""/m/02kdv5l"": ""Action"", ""/m/09zvmj"": ""Space western""}"
0,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
1,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
2,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
3,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
4,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen..."


In [5]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1t62nidmARMfM2brn2pspcivUcNCk3fqOFC7TPoBlazs#gid=0


In [11]:
import ast
# Assigning column index based on structure
title_col = 2
genres_col = -1

# Select only the title and genre columns
df_selected = df.iloc[:, [title_col, genres_col]].copy()

# Rename columns
df_selected.columns = ["Title", "Genres"]

# Convert genre dictionaries to a readable string
df_selected["Genres"] = df_selected["Genres"].apply(lambda x: ", ".join(ast.literal_eval(x).values()) if pd.notna(x) else "")


# Missing val gets replaced

df_selected.fillna("Missing", inplace=True)
# Save as CSV
df_selected.head()



,Title,Genres
0,Getting Away with Murder: The JonBenét Ramsey ...,"Mystery, Biographical film, Drama, Crime Drama"
1,Brun bitter,"Crime Fiction, Drama"
2,White Of The Eye,"Thriller, Erotic thriller, Psychological thriller"
3,A Woman in Flames,Drama
4,The Gangsters,"Short Film, Silent film, Indie, Black-and-whit..."


In [16]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# Create a "Description" column by combining Title and Genres
df_selected["Description"] = df_selected["Title"] + " " + df_selected["Genres"]
df_selected["Description"] = df_selected["Description"].fillna("")

# Compute TF-IDF on the combined text (Title + Genres)
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(df_selected["Description"])

# Function to recommend movies based on ANY text input
def recommend_movies(user_query, top_n=5):
    """
    Recommend top N movies based on any text input (not just genres).
    """
    user_tfidf = vectorizer.transform([user_query])
    cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]

    recommendations = df_selected.iloc[top_indices][["Title", "Genres"]].copy()
    recommendations["Similarity"] = cosine_similarities[top_indices]
    return recommendations

# Example: User provides a sentence as input
user_query = input("\n Enter the type of movie you want to watch: ")
recommended_movies = recommend_movies(user_query)

# Display recommendations
print("\n🔹 Top Movie Recommendations:")
print(recommended_movies.to_string(index=False))



 Enter the type of movie you want to watch: A thrilling subject

🔹 Top Movie Recommendations:
                Title                              Genres  Similarity
            Thrilling                              Comedy    0.718997
  One Thrilling Night Crime Fiction, Mystery, Comedy film    0.572757
  Subject: I Love You                 Romance Film, Drama    0.571308
          Bad Subject                         Comedy film    0.541518
The Subject Was Roses                               Drama    0.512438
